# CROP PRODUCTION ANALYSIS

...About tow or three paragraphs explaining my project

## Data Source

## Steps to Mart Creation
... Provide information about the source of the data for the mart and links to any supporting information

##  Schema Creation
    -  dim
    -  stg
    -  fact



```
/*********************************************************************/
/****************    SCHEMA DDL      ********************************/
/*******************************************************************/
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'dim' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA dim AUTHORIZATION dbo;'
END
;

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'stg' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA stg AUTHORIZATION dbo;'
END
;

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'f' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA f AUTHORIZATION dbo;'
END
;
GO
```

## Steps to Dimension and fact Creation
-  Tables Creation
    -  dim.Area
    -  dim.Item
    -  dim.Calendar
    -  f.fact